In [11]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [ ]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 'didomi_token=eyJ1c2VyX2lkIjoiMTkzNzIxM2YtNTIxOC02ZWVlLWE1M2EtY2EwNGNiYTY0OGRkIiwiY3JlYXRlZCI6IjIwMjQtMTEtMjhUMDk6MjI6MTAuODQ5WiIsInVwZGF0ZWQiOiIyMDI0LTExLTI4VDA5OjIyOjEyLjM3NFoiLCJ2ZW5kb3JzIjp7ImRpc2FibGVkIjpbImdvb2dsZSIsImM6Z29vZ2xlYW5hLTRUWG5KaWdSIiwiYzphZGV2aW50YS1tb3Rvci1tb3RvciIsImM6bWV0YXBsYXRmLU5SZVZwbUxRIiwiYzpjcml0ZW8tUDQ4ZUdUMnciLCJjOmdvb2dsZWlyZS1mZkthUGFSRCJdfSwicHVycG9zZXMiOnsiZGlzYWJsZWQiOlsiZGV2aWNlX2NoYXJhY3RlcmlzdGljcyIsImdlb2xvY2F0aW9uX2RhdGEiLCJ0cmFuc2Zlci10by1tb3RvciIsInVzb2RlbG9zLVFwTHBNOGpXIl19LCJ2ZW5kb3JzX2xpIjp7ImRpc2FibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQIzPkAQIzPkAAHABBENBRFgAAAAAAAAAAiQAAAAAAAA.YAAAAAAAAAAA; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7fX0sInNwZWNpYWxGZWF0dXJlcyI6e319; reese84=3:gDKFMKWJSXZnEuhtAgea/A==:wpqIDvQceR6CZJ1pZ9gB2rKdoV8ciIStJnJjP8qatrFYeqVuYvvZ7hdl9ODzn7ilg/VZc6TanxHqEdqJr6c87owZg22N6GTHAslFVhSj6kaHgqZHUBSpPPoHvwcDR6TFRP/TE4b1BlqDYV3a5alOYXE+WH+DLCJYkS5E35GIhiwq93t0cgLXgFc0fSVqh0+woFE0H81lowV4pHKl9ZmbbE921mjfjtuxXyBAs9ep0URv91bBhlhGp19+0IJHtHt3EeZScFvhDCXbzeNVw7GCZf5zT/ztM8sMvq0gKDDBhMm3lBSY9sSydWOLtAUiLNAST5XalWCMIBOjZhV88jOhHHQypg/LXj6Ba3tmyYKz9h8tt1Dr0/xDR1vM8QhKqY3QuS7q9Kiy5r0Oi/e52nlBLEFgm5j23ukBoThNR/yX9x/fTTTeMFSQsnTmjI5yiemWm7Fnewp3+20/gCjd89LBcw==:0jW40sc0flh/w9gJ+zuLuWSJW+7oAlrsXx4Ht9NCDRI=; cu=es-es; ASP.NET_SessionId=hdbl3akixkqnd1c3qslvlhbo; usunico=03/12/2024:14-212773; auth=hdbl3akixkqnd1c3qslvlhbo; re_uuid=9c267289-b714-4e8d-841b-bf9f1951750b',
    "Referer": "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/l",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Height": "9999",  # Establece el valor de height
    "X-Width": "1920",   # Puedes agregar otros parámetros, como width
    "X-Depth": "500",    # Ejemplo de parámetro depth, si es necesario
    "X-Color": "blue",   # Ejemplo de parámetro de color, si lo deseas
    "X-Orientation": "landscape"  # Otro ejemplo, si necesitas un parámetro de orientación
}


In [ ]:
# Crear una sesión
def check_error_page(html_page):
    # Patrón para buscar frases comunes en una página de error
    patrones_error = [
        r"sentimos la interrupciã\x93n",  # Detecta el mensaje de "Sentimos la interrupción"
    ]
    
    # Convertir el HTML a minúsculas para hacerlo insensible a mayúsculas
    html_lower = html_page.lower()
    # Buscar cada uno de los patrones de error en el HTML
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False
    
# Función para obtener los detalles del inmueble
def extract_property_info(property_data):
    # Titulo del inmueble: Se puede obtener desde la dirección del detalle
    titulo = f"{property_data['address']['city']} - {property_data['address']['district']}"

    # Precio (lo extraemos del campo 'price')
    precio = property_data['price']

    # Habitaciones: 'rooms' en las características
    habitaciones = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'rooms'), None)

    # Baños: 'bathrooms' en las características
    banos = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'bathrooms'), None)

    # Superficie: 'surface' en las características
    superficie = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'surface'), None)

    # Planta: 'floor' en las características
    planta = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'floor'), None)

    # Enlace: detalle completo
    full_link = property_data['detail'].get('es-ES', '')

    return {
        "titulo": titulo,
        "precio": precio,
        "habitaciones": habitaciones,
        "banos": banos,
        "superficie": superficie,
        "planta": planta,
        "enlace": full_link,
        'fuente': 'fotocasa',
    }

session = requests.Session()
session.headers.update(headers)


# URL base de Fotocasa
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"


# Paginación y almacenamiento de resultados
page = 1
all_listings = []
page_limit = 30
while True and page < page_limit:
    try:
        print(f"Procesando página {page}...")
        # Realizar la solicitud a la página actual
        response = session.get(f"{base_url}l/{page}")
        response.raise_for_status()  # Verificar que la solicitud sea exitosa
        error_page = check_error_page(response.text)
        if (error_page):
            print("Bloqueado por fotocasa, cambie la cookie")
            break
        if response.status_code == 200:
            # Extraer el contenido JavaScript que contiene __INITIAL_PROPS__
            match = re.search(
                r'window\.__INITIAL_PROPS__ = JSON\.parse\("((?:[^"\\]|\\.)*)"\);',
                response.text
            )
            if match:
                # Decodificar el JSON del match encontrado
                escaped_json = match.group(1)  # Esto es el JSON como string
                try:
                    # Convertir el string JSON en un diccionario de Python
                    json_data = json.loads(escaped_json.encode('utf-8').decode('unicode_escape'))
                except json.JSONDecodeError:
                    print("Error al decodificar el JSON")
            else:
                print("No se encontró la variable window.__INITIAL_PROPS__ en el HTML.")
        else:
            print(f"Error al realizar la solicitud: {response.status_code}")


        inmuebles = json_data['initialSearch']['result']['realEstates']
        for inmueble in inmuebles:
            all_listings.append(extract_property_info(inmueble))
        # # Analizar el contenido HTML
        # soup = BeautifulSoup(response.text, 'html.parser')

        # # Determinar la clase de las tarjetas en la página actual

        # class_card = "re-CardPackPremium" if page == 1 else "re-CardPackMinimal"
        # listings = soup.find_all("article", class_=class_card)


        # # Si no hay más tarjetas, salir del bucle

        # if not listings:
        #     print("No se encontraron más inmuebles. Fin de la paginación.")
        #     break


        # # Procesar cada tarjeta
        # for listing in listings:
        #     try:
        #         link_tag = listing.find('a', class_='re-CardPackPremium-carousel')
        #         href = link_tag.get('href') if link_tag else None
        #         full_link = f"https://www.fotocasa.es{href}" if href and href.startswith("/") else None

        #         titulo = listing.find("span", class_="re-I18nPropertyTitle").get_text(strip=True)
        #         precio = listing.find('span', class_="re-CardPrice").get_text(strip=True)

        #         features = listing.find("ul", class_="re-CardFeaturesWithIcons-wrapper")
        #         habitaciones = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--rooms").get_text(strip=True) if features else "N/A"
        #         banos = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--bathrooms").get_text(strip=True) if features else "N/A"
        #         superficie = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--surface").get_text(strip=True) if features else "N/A"
        #         planta = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--floor").get_text(strip=True) if features else "N/A"

        #         # Guardar datos del listado

        #         all_listings. ({
        #             "titulo": titulo,
        #             "precio": precio,
        #             "habitaciones": habitaciones,
        #             "banos": banos,
        #             "superficie": superficie,
        #             "planta": planta,
        #             "enlace": full_link,
        #             'fuente': 'fotocasa',
        #         })



        #     except Exception as e:
        #         print(f"Error procesando un listado: {e}")



        # Pasar a la siguiente página
        page += 1
        time.sleep(random.uniform(1, 8))  # Añadir un retraso para evitar bloqueos


    except Exception as e:
        print(f"Error procesando la página {page}: {e}")
        break

# Convertir los datos en un DataFrame

df = pd.DataFrame(all_listings)
df


Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Bloqueado por fotocasa, cambie la cookie


,titulo,precio,habitaciones,banos,superficie,planta,enlace,fuente
0,Murcia Capital - Norte,580 €,1.0,1,49.0,4.0,/es/alquiler/vivienda/murcia-capital/ascensor-...,fotocasa
1,La Manga del Mar Menor - Entremares,650 €,2.0,1,51.0,7.0,/es/alquiler/vivienda/la-manga-del-mar-menor/a...,fotocasa
2,Cartagena - Cartagena ciudad,800 €,2.0,1,87.0,9.0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
3,Murcia Capital - Pedanías Este,695 €,2.0,1,75.0,6.0,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
4,Murcia Capital - Centro,680 €,2.0,1,63.0,NaN,/es/alquiler/vivienda/murcia-capital/san-andre...,fotocasa
...,...,...,...,...,...,...,...,...
115,Cartagena - Pedanías Oeste,1.350 €,3.0,2,120.0,NaN,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
116,Murcia Capital - Pedanías Este,850 €,2.0,2,101.0,31.0,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
117,Cartagena - Cartagena ciudad,810 €,4.0,2,122.0,10.0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
118,Torre-Pacheco - Torre-Pacheco ciudad,670 €,2.0,1,75.0,6.0,/es/alquiler/vivienda/torre-pacheco/aire-acond...,fotocasa


In [26]:
df

,titulo,precio,habitaciones,banos,superficie,planta,enlace,fuente
0,Murcia Capital - Norte,580 €,1.0,1,49.0,4.0,/es/alquiler/vivienda/murcia-capital/ascensor-...,fotocasa
1,La Manga del Mar Menor - Entremares,650 €,2.0,1,51.0,7.0,/es/alquiler/vivienda/la-manga-del-mar-menor/a...,fotocasa
2,Cartagena - Cartagena ciudad,800 €,2.0,1,87.0,9.0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
3,Murcia Capital - Pedanías Este,695 €,2.0,1,75.0,6.0,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
4,Murcia Capital - Centro,680 €,2.0,1,63.0,NaN,/es/alquiler/vivienda/murcia-capital/san-andre...,fotocasa
...,...,...,...,...,...,...,...,...
115,Cartagena - Pedanías Oeste,1.350 €,3.0,2,120.0,NaN,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
116,Murcia Capital - Pedanías Este,850 €,2.0,2,101.0,31.0,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
117,Cartagena - Cartagena ciudad,810 €,4.0,2,122.0,10.0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
118,Torre-Pacheco - Torre-Pacheco ciudad,670 €,2.0,1,75.0,6.0,/es/alquiler/vivienda/torre-pacheco/aire-acond...,fotocasa
